In [1]:
from statsmodels.graphics.tsaplots import plot_pacf
from statsmodels.graphics.tsaplots import plot_acf
from statsmodels.tsa.statespace.sarimax import SARIMAX
from statsmodels.tsa.stattools import adfuller
from sklearn.metrics import mean_squared_error
import matplotlib.pyplot as plt
from tqdm import tqdm_notebook
import numpy as np
import pandas as pd
from itertools import product
from sklearn.metrics import *
from math import sqrt
import warnings

warnings.filterwarnings('ignore')  #tüm hataları göz ardı et

# Load your data
all_df = pd.read_csv("C:/users/FurkanBulut/Desktop/T1totalclean.csv", usecols=['Name', 'WS101']) # veri setini yükle
forecast = [] # tahminleri depolamak için boş bir liste oluştur

# Loop over your data
for i in range(0, 57601, 2): # veri setindeki her iki satır için
    data = all_df[i:i + 144] # 144 satırlık bir veri kümesi oluştur

    def optimize_SARIMA(endog, parameters_list, d, D, s): # SARIMA modelini optimize et
        """
            Return dataframe with parameters and corresponding AIC

            endog - the observed variable
            parameters_list - list with (p, q, P, Q) tuples
            d - integration order in ARIMA model
            D - seasonal integration order
            s - length of season
        """
        results = [] # sonuçları depolamak için boş bir liste oluştur
        for param in tqdm_notebook(parameters_list): # parametrelerin her biri için
            try: # hata oluşursa devam et 
                model = SARIMAX(endog, 
                                order=(param[0], d, param[1]), seasonal_order=(param[2], D, param[3], s), 
                                simple_differencing=False).fit(disp=False) # SARIMA modelini kur
            except: # hata oluşursa devam et
                continue 
            aic = model.aic # modelin aic değerini hesapla. aic değeri ne kadar düşükse model o kadar iyi. Aic: Akaike Information Criterion yani Akaike Bilgi Kriteri
            results.append([param, aic]) # sonuçları listeye ekle

        result_df = pd.DataFrame(results)  # sonuçları dataframe'e dönüştür
        result_df.columns = ['(p,q)x(P,Q)', 'AIC'] # sütun isimlerini değiştir
        # Sort in ascending order, lower AIC is better 
        result_df = result_df.sort_values(by='AIC', ascending=True).reset_index(drop=True) # aic değerlerine göre sırala
        return result_df # sonuçları döndür

    p = range(0, 3, 1) # p, q, P, Q değerlerini belirle
    d = 1            # d ve D değerlerini belirle
    q = range(0, 3, 1) # p, q, P, Q değerlerini belirle
    P = range(0, 3, 1) # p, q, P, Q değerlerini belirle
    D = 1           # d ve D değerlerini belirle
    Q = range(0, 3, 1) # p, q, P, Q değerlerini belirle
    s = 4          # s değerini belirle
    parameters = product(p, q, P, Q) # tüm kombinasyonları oluştur
    parameters_list = list(parameters)  # kombinasyonları listeye dönüştür

    result_df = optimize_SARIMA(data['WS101'], parameters_list, 1, 1, 4) # optimize_SARIMA fonksiyonunu çağır
    result_df_min = result_df[result_df.AIC == result_df.AIC.min()] # aic değeri en düşük olanı seç
    p = result_df_min['(p,q)x(P,Q)'][0][0] # seçilen parametreleri al
    q = result_df_min['(p,q)x(P,Q)'][0][1] # seçilen parametreleri al
    P = result_df_min['(p,q)x(P,Q)'][0][2] # seçilen parametreleri al
    Q = result_df_min['(p,q)x(P,Q)'][0][3] # seçilen parametreleri al
    best_model = SARIMAX(data['WS101'], order=(p, d, q), seasonal_order=(P, D, Q, s), simple_differencing=False) # en iyi modeli kur
    res = best_model.fit(disp=False) # modeli eğit
    n_forecast = 2 # tahmin edilecek değer sayısı
    predict = res.get_prediction(end=best_model.nobs + n_forecast) # tahmin et
    prediction = predict.predicted_mean[-n_forecast:] # tahmin edilen değerleri al
    forecast.append(prediction) # tahmin edilen değerleri listeye ekle

df = all_df[144:57601] # gerçek değerleri al
df1 = pd.DataFrame(np.concatenate(forecast)) # tahmin edilen değerleri dataframe'e dönüştür
expected = df['WS101'] # gerçek değerleri al
prediction = df1 # tahmin edilen değerleri al
MSE = mean_squared_error(expected, prediction) # hata kareler ortalaması
R2 = r2_score(expected, prediction) # r kare
RMSE = sqrt(mean_squared_error(expected, prediction))   # hata kareler ortalamasının karekökü
MAE = mean_absolute_error(expected, prediction) # mutlak hata
 
print("MSE:", MSE)  # hata kareler ortalaması
print("R2:", R2) # r kare 
print("RMSE:", RMSE) # hata kareler ortalamasının karekökü
print("MAE:", MAE) # mutlak hata


SyntaxError: invalid syntax (3399344029.py, line 1)